In [ ]:
import yaml
with open('samples.yml', 'r') as f:
    samples = yaml.safe_load(f)

templates = '''campaign:
  name: %NAME%
  crab_template: ../test/template_crab.py

  # User specific
  workArea: %NAME%
  storageSite: T1_DE_KIT_Disk
  outLFNDirBase: /store/user/XXX/PFNano
  voGroup: dcms # or leave empty

  # Campaign specific
  tag_extension: %TAG% # Will get appended after the current tag
  tag_mod: # Will modify name in-place for MC eg. "PFNanoAODv1" will replace MiniAODv2 -> PFNanoAODv1
  # If others shall be able to access dataset via DAS (important when collaborating for commissioning!)
  publication: True
  config: ../test/%CONFIG%
  # Specify if running on data
  data: %DATA%
  lumiMask: %LUMIMASK% 
  # datasets will take either a list of DAS names or a text file containing them
  # do NOT submit too many tasks at the same time, despite it looking more convenient to you
  # wait for tasks to finish before submitting entire campaigns,
  # it's better to request one dataset at a time (taking fairshare into account)
  datasets: |
    %DATASETS%
'''

def get_config(campaign, is_noPF=False):
    config_prefix = {
        'mc_summer22': 'nano_mc_Run3',
        'mc_summer22EE': 'nano_mc_Run3_EE',
        'data_2022CDE_rereco': 'nano_data_2022CDE',
        'data_2022FG_prompt': 'nano_data_2022FG',
    }
    return config_prefix[campaign] + ('_noPF' if is_noPF else '') + '_NANO.py'

tag_extension = 'BTV_Run3_2022_Comm_v2'

for campaign in ['mc_summer22', 'mc_summer22EE', 'data_2022CDE_rereco', 'data_2022FG_prompt']:
    for sam in samples[campaign]:
        if sam in ['qcdmu', 'qcdincl', 'BTagMu', 'JetMET']:
            is_noPF = False
        else:
            is_noPF = True
        name = f'{campaign}_{sam}' + ('_noPF' if is_noPF else '')
        templates_write = templates\
            .replace('%NAME%', name)\
            .replace('%TAG%', tag_extension)\
            .replace('%CONFIG%', get_config(campaign, is_noPF))\
            .replace('%DATA%', 'True' if campaign.startswith('data_') else 'False')\
            .replace('%LUMIMASK%', 'jsons/Cert_Collisions2022_355100_362760_Golden.json' if campaign.startswith('data_') else '')\
            .replace('%DATASETS%', '\n    '.join(samples[campaign][sam]))
        with open(f'crab_ymls/{name}.yml', 'w') as f:
            f.write(templates_write)
        # break